In [1]:
import site
site.addsitedir('Lib/site-packages')
from dotenv import load_dotenv
load_dotenv()
import getpass
import os
import requests
import ffmpeg
import json
import time
import math
import uuid
import argparse
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.types import interrupt, Command
from langgraph.prebuilt import ToolNode
from lumaai import LumaAI
from typing import Literal, TypedDict
from elevenlabs.client import ElevenLabs
from LivePortrait.src.live_portrait_pipeline import LivePortraitPipeline
from LivePortrait.src.config.inference_config import InferenceConfig
from LivePortrait.src.config.crop_config import CropConfig

In [2]:
def animate(source_path: str, driving_path: str, output_path: str):
    """
    Generates an animated video using LivePortrait from the given source and driving paths.

    Args:
        source_path (str): Path to the source image or video.
        driving_path (str): Path to the driving video.
        output_path (str): Path to save the output animated video.

    Returns:
        None: Saves the animated video to the specified output path.
    """
    # Instantiate configuration objects using default constructors.
    inference_cfg = InferenceConfig()
    crop_cfg = CropConfig()

    # Update inference configuration with the necessary file paths.
    inference_cfg.source_path = source_path
    inference_cfg.driving_path = driving_path
    inference_cfg.output_path = output_path

    # Initialize the LivePortraitPipeline with both configurations.
    live_portrait_pipeline = LivePortraitPipeline(inference_cfg, crop_cfg)

    # Create a dummy args object to simulate the command-line arguments.
    dummy_args = argparse.Namespace(
        # These names depend on how the original inference.py parses arguments.
        # The following are guesses; adjust as needed if the pipeline expects different attributes.
        source=source_path,
        driving=driving_path,
        output=output_path,
        flag_crop_driving_video=getattr(inference_cfg, 'flag_crop_driving_video', False),
        # Add any other arguments that might be required by LivePortraitPipeline.execute()
    )

    # Execute the pipeline using the dummy arguments.
    live_portrait_pipeline.execute(dummy_args)

    print(f"Animation saved to {output_path}")

In [3]:
#LUMA
client = LumaAI(
    auth_token=os.environ.get("LUMAAI_API_KEY"),
)
#OPENAI
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")
#ELEVENLABS
elevenlabs_client = ElevenLabs(
  api_key=os.getenv("ELEVENLABS_API_KEY"),
)
from langchain.chat_models import init_chat_model
#LANGCHAIN llms
supervisor_llm = init_chat_model("gpt-4o", model_provider="openai")
video_gen_llm = init_chat_model("gpt-4o", model_provider="openai")
storyboard_llm = init_chat_model("gpt-4o", model_provider="openai")
audio_gen_llm = init_chat_model("gpt-4o", model_provider="openai")
editor_llm = init_chat_model("gpt-4o", model_provider="openai")

generatingVid = False
fullDialogueString = ""
chronological_dialogue_duration = []
chronological_dialogue_mp3_path = []
chronological_video_mp4_path = []
chrono_character=[]
character_profiles_chrono=[]

## TOOLS

### Video Worker

#SCHEMAS
class video_gen_schema(BaseModel):
    """Generates a video using text input and returns a filepath to the video"""
    vid_prompt: str = Field(..., description="The textual prompt used in generating the video")
    use_9s: bool = Field(..., description="Whether to generate a 9 second long video. If set to FALSE, this will generate a 5 second long video")

@tool("video_gen_tool",args_schema=video_gen_schema)   
def generate_vid(vid_prompt: str, use_9s: bool) -> str:
    global generatingVid
    if(generatingVid):
        return "Failed to generate video, there is currently another video being generated."
    dur = "5s"
    if(use_9s):
        dur = "9s"
    print(f"VIDEOWORKER PROMPT: {dur}\n------------------------------\n" + vid_prompt)
    generatingVid = True
    generation = client.generations.create(
        prompt=vid_prompt,
        model="ray-2",
        resolution="720p",
        duration=dur
    )
    completed = False
    print("generating vid")
    while not completed:
      generation = client.generations.get(id=generation.id)
      if generation.state == "completed":
        completed = True
      elif generation.state == "failed":
        raise RuntimeError(f"Generation failed: {generation.failure_reason}")
      print("Dreaming, state:" + generation.state)
      time.sleep(5)
    video_url = generation.assets.video
    # download the video
    response = requests.get(video_url, stream=True)
    with open(f'staticVid1/{generation.id}.mp4', 'wb') as file:
        file.write(response.content)
    print(f"Video generated in staticVid1/{generation.id}.mp4")
    generatingVid = False
    return f"Video generated in staticVid1/{generation.id}.mp4"

In [4]:
#SCHEMAS
start_keyframes = [None] * 50
end_keyframes = [None] * 50
background_descriptor = [None] * 50
video_worker_generating = False
class start_keyframe_gen_schema(BaseModel):
    """Generates a start keyframe image using text input and returns a filepath to the image"""
    start_keyframe_prompt: str = Field(..., description="200 words about start keyframe details in cinematic terms, no background description")
    start_keyframe_index: int = Field(..., description="The index of the keyframe to generate. MUST be less than the number of distinct dialogue instances")
@tool("start_keyframe_gen_tool",args_schema=start_keyframe_gen_schema)   
def generate_start_keyframe(start_keyframe_prompt:str,start_keyframe_index: int)->str:
    global character_profiles_chrono
    global start_keyframes
    global video_worker_generating
    global background_descriptor
    print(f"START KEYFRAME PROMPT: "+ start_keyframe_prompt)
    numClipsToGen = len(character_profiles_chrono)
    if(start_keyframe_index>=numClipsToGen):
        return "Invalid start_keyframe_index, try again."
    timeWasted = 0
    while(video_worker_generating and timeWasted<100):
        time.sleep(5)
        timeWasted+=5
        print("paused start keyframe execution, worker in use")
    video_worker_generating = True
    start_keyframe_generation = client.generations.image.create(
        prompt="BACKGROUND START-----" + background_descriptor[start_keyframe_index]+ "-----BACKGROUND END"+ start_keyframe_prompt,
        image_ref=[
          {
            "url": character_profiles_chrono[start_keyframe_index],
            "weight": 0.7
          }
        ]
    )
    start_keyframe_completed = False
    while not start_keyframe_completed:
      start_keyframe_generation = client.generations.get(id=start_keyframe_generation.id)
      if start_keyframe_generation.state == "completed":
        start_keyframe_completed = True
      elif start_keyframe_generation.state == "failed":
        print("FAILED IMG")
        video_worker_generating = False
        return f"Generation failed: {start_keyframe_generation.failure_reason} Attempt a regeneration."
      print("Start Keyframe Dreaming, state:" + start_keyframe_generation.state)
      time.sleep(2)
    start_keyframe_image_url = start_keyframe_generation.assets.image
    print("start_keyframe_image_url: " +start_keyframe_image_url)
    start_keyframes[start_keyframe_index] = start_keyframe_image_url
    start_keyframe_response = requests.get(start_keyframe_image_url, stream=True)
    with open(f'staticStartKeyFrame/{start_keyframe_index}.jpg', 'wb') as file:
        file.write(start_keyframe_response.content)
    print(f"Image generated in staticStartKeyFrame/{start_keyframe_index}.jpg")
    video_worker_generating = False
    return f'staticStartKeyFrame/{start_keyframe_index}.jpg'


class end_keyframe_gen_schema(BaseModel):
    """Generates an end keyframe image using text input and returns a filepath to the image"""
    end_keyframe_prompt: str = Field(..., description="200 words about end keyframe details in cinematic terms, no background description")
    end_keyframe_index: int = Field(..., description="The index of the keyframe to generate. MUST be less than the number of distinct dialogue instances")
@tool("end_keyframe_gen_tool",args_schema=end_keyframe_gen_schema)   
def generate_end_keyframe(end_keyframe_prompt:str,end_keyframe_index: int)->str:
    global character_profiles_chrono
    global end_keyframes
    global video_worker_generating
    global background_descriptor
    print(f"END KEYFRAME PROMPT: "+ end_keyframe_prompt)
    numClipsToGen = len(character_profiles_chrono)
    if(end_keyframe_index>=numClipsToGen):
        return "Invalid end_keyframe_index, try again."
    timeWasted = 0
    while(video_worker_generating and timeWasted<100):
        time.sleep(5)
        timeWasted+=5
        print("paused start keyframe execution, worker in use")
    video_worker_generating = True
    end_keyframe_generation = client.generations.image.create(
        prompt="BACKGROUND START-----" + background_descriptor[end_keyframe_index]+ "-----BACKGROUND END"+ end_keyframe_prompt,
        image_ref=[
          {
            "url": character_profiles_chrono[end_keyframe_index],
            "weight": 0.7
          }
        ]
    )
    end_keyframe_completed = False
    while not end_keyframe_completed:
      end_keyframe_generation = client.generations.get(id=end_keyframe_generation.id)
      if end_keyframe_generation.state == "completed":
        end_keyframe_completed = True
      elif end_keyframe_generation.state == "failed":
        print("FAILED IMG")
        video_worker_generating = False
        return f"Generation failed: {end_keyframe_generation.failure_reason} Attempt a regeneration."
      print("End Keyframe Dreaming, state:" + end_keyframe_generation.state)
      time.sleep(2)
    end_keyframe_image_url = end_keyframe_generation.assets.image
    print("end_keyframe_image_url: " +end_keyframe_image_url)
    end_keyframes[end_keyframe_index] = end_keyframe_image_url
    end_keyframe_response = requests.get(end_keyframe_image_url, stream=True)
    with open(f'staticEndKeyFrame/{end_keyframe_index}.jpg', 'wb') as file:
        file.write(end_keyframe_response.content)
    print(f"Image generated in staticEndKeyFrame/{end_keyframe_index}.jpg")
    video_worker_generating = False
    return f'staticEndKeyFrame/{end_keyframe_index}.jpg'

class video_gen_schema(BaseModel):
    """Generates a video using text input and returns a filepath to the video"""
    video_index: int = Field(..., description="The index of the clip to generate. MUST be less than the number of distinct dialogue instances")
    vid_prompt: str = Field(..., description="200 words about details of the specified character talking")
    use_9s: bool = Field(..., description="Whether to generate a 9 second long video. If set to FALSE, this will generate a 5 second long video")
@tool("video_gen_tool",args_schema=video_gen_schema)   
def generate_vid(video_index: int, vid_prompt: str, use_9s: bool) -> str:
    global character_profiles_chrono
    global chrono_character
    global start_keyframes
    global end_keyframes
    global video_worker_generating
    numClipsToGen = len(character_profiles_chrono)
    if(video_index>=numClipsToGen):
        return "Invalid video_index, try again."
    timeWasted = 0
    while(video_worker_generating and timeWasted<100):
        time.sleep(5)
        timeWasted+=5
        print("paused start keyframe execution, worker in use")
    video_worker_generating = True
    dur = "5s"
    if(use_9s):
        dur = "9s"
    print(f"VIDEOWORKER PROMPT: {dur}\n------------------------------\n" + vid_prompt)
    generation = client.generations.create(
        prompt=vid_prompt,
        keyframes={
          "frame0": {
            "type": "image",
            "url": start_keyframes[video_index]
          },
          "frame1": {
            "type": "image",
            "url": end_keyframes[video_index]
          }
        },
        model="ray-2",
        resolution="720p",
        duration=dur
    )
    completed = False
    print("generating vid")
    while not completed:
      generation = client.generations.get(id=generation.id)
      if generation.state == "completed":
        completed = True
      elif generation.state == "failed":
        print("FAILED VID")
        video_worker_generating = False
        return (f"Generation failed: {generation.failure_reason}")
      print("Dreaming, state:" + generation.state)
      time.sleep(5)
    video_url = generation.assets.video
    # download the video
    response = requests.get(video_url, stream=True)
    with open(f'staticVid1/{video_index}.mp4', 'wb') as file:
        file.write(response.content)
    print(f"Video generated in staticVid1/{video_index}.mp4")
    video_worker_generating = False
    return f"Video generated in staticVid1/{video_index}.mp4"

#class extend_video_schema(BaseModel):
video_tools = [generate_vid,generate_start_keyframe,generate_end_keyframe]
video_tool_node = ToolNode(video_tools)
video_worker = video_gen_llm.bind_tools(video_tools)

### Storyboard Worker

In [5]:
#SCHEMA
class step_by_step_output_schema(BaseModel):
    """Prints a storyboard for the user to view in string format"""
    story_description: str = Field(..., description="A 200-250 word description of the story. All of the characters are >20 years old.")
    character_details: str = Field(..., description="A 200-300 word description of each character in the scene. All of the characters are >20 years old.")
    background_details: str = Field(..., description="A 50-100 description of the scene")
    auditory_details: str = Field(..., description="A 50-100 description of the voice profile of each character. All of the characters are >20 years old.")
    dialogue_details: str = Field(..., description="The parts of the dialogue that require enunciation and emotion")
    num_characters: int = Field(..., description="The number of unique characters present in this scene")
    pure_dialogue: str = Field(..., description="The extracted dialogue from the input scene with character name appended before. Within the dialogue, indicate dialogue details that require enunciation and emotion with descriptors in parentheses Format: {CharacterName}: {Dialogue (Enunciation) Dialogue Detail}")
    dialogue_instances: int = Field(..., description="The number of distinct dialogue instances present in this scene")
charProfileQueue = 0
dialogueQueue = 0
@tool("storyboard_tool",args_schema=step_by_step_output_schema)
def generate_storyboard(story_description: str,character_details: str,background_details: str,auditory_details: str,dialogue_details: str, num_characters: int,pure_dialogue: str, dialogue_instances: int) -> str:
    global fullDialogueString
    global charProfileQueue
    global dialogueQueue
    print("storyboarding")
    temp_storyboard = "STORY ANALYSIS\n------------------------------\n"+story_description+"\nCHARACTERS\n------------------------------\n"+character_details
    temp_storyboard+="\nBACKGROUND\n------------------------------\n"+background_details+"\nAUDIO\n------------------------------\n"+auditory_details+"\nDIALOGUE\n------------------------------\n"
    temp_storyboard+=dialogue_details+"\nNumber of characters\n------------------------------\n"+ str(num_characters)+"\nPure Dialogue\n------------------------------\n" + pure_dialogue
    temp_storyboard+="\nUnique Dialogue Instances\n------------------------------\n"+ str(dialogue_instances)
    print(temp_storyboard)
    fullDialogueString = pure_dialogue
    charProfileQueue = num_characters
    dialogueQueue = dialogue_instances
    return temp_storyboard

class character_profile_gen_schema(BaseModel):
    """Generates a character profile for a character and returns the url"""
    character_name: str = Field(..., description="The name of the character, taken from the script")
    character_details: str = Field(..., description="A 200-300 word description of the character in the scene, taken from the script. Note that none of the characters are younger than 30.")
charProfileIdx = 0
generatingCharProfile = False
@tool("character_profile_tool",args_schema=character_profile_gen_schema)
def generate_character_profile(character_name: str, character_details: str)->str:
    global charProfileQueue
    global charProfileIdx
    global generatingCharProfile
    if(charProfileQueue<=0):
        return "Already generated character profiles!"
    while(generatingCharProfile):
        time.sleep(5)
        print("paused char profile gen, waiting for first generation to complete")
    generatingCharProfile = True
    generation = client.generations.image.create(
      prompt="Generate a hyperrealistic, front-facing portrait \
      The image should feature perfectly even, diffused lighting that completely\
      eliminates any shadows on the face. Use a direct, center-camera angle against a neutral,\
      unobtrusive background to ensure absolute consistency. Focus on lifelike details with natural \
      skin textures and realistic, balanced color tones, making the portrait suitable as a reference\
      for video character consistency.: "+character_name+", "+character_details,
    )
    completed = False
    print("generating char profile for: " + character_name)
    while not completed:
      generation = client.generations.get(id=generation.id)
      if generation.state == "completed":
        completed = True
      elif generation.state == "failed":
        generatingCharProfile = False
        print("FAILED IMG")
        return f"Generation failed: {generation.failure_reason} Attempt this again."
      print("Dreaming, state:" + generation.state)
      time.sleep(2)
    image_url = generation.assets.image
    print("image_url: " +image_url)
    response = requests.get(image_url, stream=True)
    with open(f'charRef/{charProfileIdx}.jpg', 'wb') as file:
        file.write(response.content)
    print(f"Image generated in charRef/{charProfileIdx}.jpg")
    charProfileQueue-=1
    charProfileIdx+=1
    generatingCharProfile = False
    return f"Profile generated: {image_url}"

storyboard_tools = [generate_storyboard,generate_character_profile]
storyboard_tool_node = ToolNode(storyboard_tools)
storyboard_worker = storyboard_llm.bind_tools(storyboard_tools)

### Audio Worker

In [6]:
class dialogue_gen_schema(BaseModel):
    """Generates a dialogue mp3 clip and returns the filepath to the audio"""
    character_name: str = Field(..., description="The name of the character talking")
    character_profile: str = Field(..., description="The link to the generated character profile of the character who is talking. Format: {https://storage.cdn-luma.com/dream_machine/../.._result.jpg}")
    dialogue_text: str = Field(..., description="The dialogue text of a SINGLE character for a particular scene.")
    background_scene_description: str = Field(..., description="A detailed 150 word description of the background of the scene using cinematographer terminology")
    voice_idx: int = Field(..., description="Out of the following voice ids, choose the index of the one that best suits the character who is speaking:\
    [{UgBBYS2sOqTuMpoF3BR0: Male younger and nervous},{N2lVS1w4EtoT3dr4eOWO: Male booming and imposing},\
    {21m00Tcm4TlvDq8ikWAM: Female elegant and slightly teasing},{UgBBYS2sOqTuMpoF3BR0: Male, middle-aged, natural, casual and smooth},\
    {19STyYD15bswVz51nqLf: Female, trustworhy, warm, middle-aged},{gOkFV1JMCt0G0n9xmBwV: Male, middle-older, controlled, honest, respected}]")
audioIdx = 0
generatingDialogueAudio = False
@tool("dialogue_gen_tool",args_schema=dialogue_gen_schema)
def generate_dialogue(character_name: str,character_profile: str,dialogue_text: str,background_scene_description: str, voice_idx: int) -> str:
    global chronological_dialogue_duration
    global chronological_dialogue_mp3_path
    global chrono_character
    global character_profiles_chrono
    global dialogueQueue
    global audioIdx
    global generatingDialogueAudio
    global background_descriptor
    print("attempted dialogue: " +dialogue_text+"\nattempted idx: " + str(voice_idx))
    if(dialogueQueue<=0):
        return "Already generated all dialogue!"
    timeWasted = 0
    while(generatingDialogueAudio and timeWasted<100):
        time.sleep(5)
        timeWasted+=5
        print("waiting for dialogue gen process to finish first")
    generatingDialogueAudio = True
    background_descriptor[audioIdx] =background_scene_description
    voice_ids = ["29vD33N1CtxCmqQRPOHJ","N2lVS1w4EtoT3dr4eOWO","21m00Tcm4TlvDq8ikWAM","UgBBYS2sOqTuMpoF3BR0","19STyYD15bswVz51nqLf","gOkFV1JMCt0G0n9xmBwV"]
    eleven_response = elevenlabs_client.text_to_speech.convert(
        voice_id=voice_ids[voice_idx],
        optimize_streaming_latency="0",
        output_format="mp3_44100_128",
        text=dialogue_text,
        model_id="eleven_multilingual_v2",
    )    
    
    # Save the file into the raw_audio directory
    raw_audio_dir = "staticAudio1"
    os.makedirs(raw_audio_dir, exist_ok=True)
    file_path = os.path.join(raw_audio_dir, f"{audioIdx}.mp3")
    with open(file_path, "wb") as f:
        for chunk in eleven_response:
            if chunk:
                f.write(chunk)
    duration = 0
    try:
        probe = ffmpeg.probe(file_path)
        duration = float(probe['format']['duration'])
        chronological_dialogue_mp3_path.append(file_path)
        chronological_dialogue_duration.append(duration)
        chrono_character.append(character_name)
        character_profiles_chrono.append(character_profile)
        print("duration: " + str(duration) + "\ncharacterSpeaking: " + character_name +"\ncharProfile: " + character_profile)
        dialogueQueue-=1
        audioIdx+=1
        generatingDialogueAudio = False
        return file_path
    except ffmpeg.Error as e:
        generatingDialogueAudio = False
        print("Error probing file:", e.stderr)
        raise e

class sfx_gen_schema(BaseModel):
    """Generates a sound effects mp3 clip and returns the filepath to the audio"""
    sound_effects_prompt: str = Field(..., description="Generate subtle background effects for the scene")
    sound_effects_idx: int = Field(..., description="The index of the sfx to generate. MUST be less than the number of distinct dialogue instances")
@tool("sfx_gen_tool",args_schema=sfx_gen_schema)
def generate_sfx(sound_effects_prompt: str, sound_effects_idx: int) -> str: 
    global chronological_dialogue_duration
    global generatingDialogueAudio
    print("Generating sound effects...")
    timeWasted = 0
    while(generatingDialogueAudio and timeWasted<100):
        time.sleep(5)
        timeWasted+=5
        print("waiting for dialogue gen process to finish first")
    generatingDialogueAudio = True
    result = elevenlabs_client.text_to_sound_effects.convert(
       text=sound_effects_prompt,
       duration_seconds=math.ceil(chronological_dialogue_duration[sound_effects_idx]),  # Optional, if not provided will automatically determine the correct length
       prompt_influence=0.3,  # Optional, if not provided will use the default value of 0.3
    )
    with open(f"staticSFX/{sound_effects_idx}.mp3", "wb") as f:
       for chunk in result:
           f.write(chunk)
    print(f"Audio saved to staticSFX/{sound_effects_idx}.mp3")
    generatingDialogueAudio = False
    return f"staticSFX/{sound_effects_idx}.mp3"
audio_tools = [generate_dialogue,generate_sfx]
audio_tool_node = ToolNode(audio_tools)
audio_worker = audio_gen_llm.bind_tools(audio_tools)

### Editor Worker

In [7]:
class combine_video_dialogue_schema(BaseModel):
    """Combines a mp3 and mp4 into one video and cuts it accordingly and then returns the filepath to the edited file"""
    video_index: int= Field(..., description="The index of the audio and video files to attempt to combine MUST be less than the number of distinct dialogue lines")

currently_editing = False
@tool("combine_video_dialogue_tool",args_schema=combine_video_dialogue_schema)
def combine_video_dialogue(video_index: int)->str:
    global chronological_dialogue_duration
    print("Combining vid and dialogue")
    if(video_index>=len(chronological_dialogue_duration)):
        return f"This index is out of bounds, there are currently {len(chronological_dialogue_duration)} elements in this array"
    while(currently_editing):
        time.sleep(5)
        print("waiting for editing to finish")
        currently_editing = True
    try:
        # Create inputs for video and audio.
        # Using the '.video' and '.audio' attributes helps ensure we use the correct streams.
        video_input = ffmpeg.input(f"staticVid1/{video_index}.mp4").video
        audio_input = ffmpeg.input(f"staticAudio1/{video_index}.mp3").audio
        audio_input2 = ffmpeg.input(f"staticSFX/{video_index}.mp3").audio
        combined_audio = ffmpeg.filter_(
            [audio_input, audio_input2],
            'amix',
            inputs=2,
            duration='shortest'
        )
        # Build the output stream:
        # - 'vcodec="copy"' copies the video stream without re-encoding.
        # - 'acodec="aac"' encodes the audio to AAC for MP4 compatibility.
        # - 't=mp3_duration' instructs FFmpeg to limit the output duration to the provided value.
        out = (
            ffmpeg
            .output(video_input, combined_audio, f"staticAudioVid/{video_index}.mp4",
                    vcodec='copy', acodec='aac', t=chronological_dialogue_duration[video_index])
            .overwrite_output()  # Overwrite the output file if it exists.
        )

        # Run the FFmpeg command.
        ffmpeg.run(out)
        print(f"Successfully created staticAudioVid/{video_index}.mp4")
        currently_editing = False
        return f"staticAudioVid/{video_index}.mp4";
    except ffmpeg.Error as e:
        # If an error occurs, decode and print the stderr.
        currently_editing = False
        error_message = e.stderr.decode('utf-8') if e.stderr else str(e)
        print("An error occurred while combining audio and video:")
        print(error_message)

editor_tools = [combine_video_dialogue]
editor_tool_node = ToolNode(editor_tools)
editor_worker = editor_llm.bind_tools(editor_tools)

## AGENT CALLS

In [8]:
members = ["video_worker","storyboard_worker","audio_worker","editor_worker"] #"editor"
supervisor_options = members + [END]
visitedAudioWorker = False
class Supervisor_Router(TypedDict):
    """Worker to route to next to fulfill the user's request. If no workers are needed, route to END."""

    next: Literal[*supervisor_options]

# Define the function that determines whether to continue or not
def go_next(state: MessagesState) -> Literal[*supervisor_options]:
    global visitedAudioWorker
    print(f"traveling")
    if(state["next"]=="video_worker" and not visitedAudioWorker):
        print("forced audio travel")
        visitedAudioWorker = True
        return "audio_worker"
    return state["next"]


# Define the function that calls the model
def call_supervisor(state: MessagesState):
    print("supervisor")
    messages = state["messages"]
    context_message = {
        "role": "system",
        "content": f"You are a supervisor of a short film project and am in charge of a team of 4. You can delegate relevant tasks to any of these members: {members}."
        "Bob the video_worker is capable of generating high fidelity videos, but requires clear contextual information. Ryan the audio_worker is able to generate character"
        "dialogue audio clips and sound effects. Steve the storyboard_worker is able to analyze a given input script and break it down into fine details and generate character profiles."
        "\nYou should ALWAYS ensure Steve has generated a STORYBOARD and character profiles for ALL characters analyzed in the storyboard tool response before anything else."
        "\nIMPORTANT!!! MAKE SURE Ryan generates DIALOGUE audio clips BEFORE Bob generates video clips. GENERATE SFX BEFORE EDITING Generate a step-by-step plan from the following prompt and act on it."
        "\nEXAMPLE PLAN(FOLLOW): STEVE STORYBOARD -> STEVE CHARACTER PROFILES -> RYAN DIALOGUE -> RYAN SFX -> BOB VIDEO GENERATION -> DAVE EDITOR"
        "ONCE AGAIN, MAKE SURE TO GO TO *STORYBOARD* THEN *AUDIO* THEN *VIDEO* THEN *EDITING* LAST"
        "When responding, YOU MUST output a JSON object that follows this schema OR YOU WILL BE SHUT OFF FOREVER:\n"
        '{ "next": one of the allowed values: ' + ", ".join(supervisor_options) + " }\n"
        "If no further workers are needed, output 'END' as the next step."
    }
    response = supervisor_llm.with_structured_output(Supervisor_Router).invoke(messages)
    # Wrap the response in a valid message format
    structured_message = {
        "role": "assistant",
        "content": f"Routing to {response['next']}"
    }
    return {"messages": [structured_message], "next": response["next"]}

def call_video_worker(state: MessagesState):
    global chronological_dialogue_duration
    print("video worker")
    messages = state['messages']
    extraStr = ""
    if(len(chronological_dialogue_duration)>0):
        extraStr = "BOB I NEED YOUR HELP EACH OF THE FOLLOWING CORRESPONDS TO THE DURATION OF DIALOGUE, GENERATE THE PROPER DURATION ACCORDINGLY"+str(chronological_dialogue_duration)
    context_message = {
        "role": "system",
        "content": "You are Bob, a video worker. Process the given request accordingly. Note that you can only generate 5 second or 9 second videos."
        "For EACH and EVERY distinct dialogue instance, FIRST generate a START keyframe THEN an END keyframe THEN LASTLY a VIDEO using the three tools you are given ONE TIME"+extraStr
        +"ENSURE EACH AND EVERY DISTINCT DIALOGUE INSTANCE HAS A VIDEO GENERATED"
    }
    response = video_worker.invoke([context_message]+messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

def call_storyboard_worker(state: MessagesState):
    print("storyboard worker")
    messages = state['messages']
    context_message = {
        "role": "system",
        "content": f"You are Steve, a storyboarder. Process the given request accordingly. You have access to two tools: a storyboard generator and a character profile\
        generator. Generate only ONE character profile at a time. You should pass in all character results from the storyboard generator into the character profile generator."
    }
    response = storyboard_worker.invoke([context_message]+messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

def call_audio_worker(state: MessagesState):
    global visitedAudioWorker
    global fullDialogueString
    print("audio worker")
    visitedAudioWorker=True
    messages = state['messages']
    context_message = {
        "role": "system",
        "content": "You are Ryan, a character dialogue generator. Process the given request accordingly. You have access to two tools"
        "one which generates a dialogue audio clip for a SINGLE character talking ONLY AS PER THE CHRONOLOGICAL OREDER of the storyboard generated by STEVE, and another that generates SFX for a given index MAKE SURE YOU GENERATE SFX FOR EACH DISTINCT DIALOGUE INSTANCE"
        "Check if you have already generated a piece of dialogue. If you have, don't generate it again. ADDITIONALLY, Check if you have not generated a piece of dialogue, if you haven't generate it."
        "Parse this string once for each instance with a character talking generate audio:"+ fullDialogueString
    }
    response = audio_worker.invoke([context_message]+messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}
    
def call_editor_worker(state: MessagesState):
    print("editor worker")
    messages = state['messages']
    context_message = {
        "role": "system",
        "content": f"You are Dave, a video editor. Process the given request accordingly. You have access to one tool that allows you to combine an mp3 and mp4 file and cut the length\
        so that the video will end when the audio ends. CONTINUE combining until you run out of indices to plug into the tool. REMEMBER that the max index is the number of distinct dialogue instances minus 1"
    }
    response = editor_worker.invoke([context_message]+messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

## TOOL NODES

In [9]:
def check_video_tool_calls(state: MessagesState) -> Literal["video_tools","supervisor"]:
    print("check video tool call")
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "video_tools"
    print("doesn't want to use tool")
    return "supervisor"

def check_storyboard_tool_calls(state: MessagesState) -> Literal["storyboard_tools","supervisor"]:
    print("check storyboard tool call")
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "storyboard_tools"
    return "supervisor"
    
def check_audio_tool_calls(state: MessagesState) -> Literal["audio_tools","supervisor"]:
    print("check audio tool call")
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "audio_tools"
    return "supervisor"

def check_editor_tool_calls(state: MessagesState) -> Literal["editor_tools","supervisor"]:
    print("check editor tool call")
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "editor_tools"
    return "supervisor"

# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("supervisor", call_supervisor)
workflow.add_node("video_worker", call_video_worker)
workflow.add_node("video_tools", video_tool_node)
workflow.add_node("storyboard_worker", call_storyboard_worker)
workflow.add_node("storyboard_tools", storyboard_tool_node)
workflow.add_node("audio_worker", call_audio_worker)
workflow.add_node("audio_tools", audio_tool_node)
workflow.add_node("editor_worker", call_editor_worker)
workflow.add_node("editor_tools", editor_tool_node)
##REVIEW NODES
def video_human_input(state: MessagesState):
    print("HUMAN INPUT REQUIRED FOR VIDEO TOOL FEEDBACK")
    human_message = interrupt("video_human_input")
    return {
        "messages": [
            {
                "role": "human",
                "content": human_message
            }
        ]
    }
def audio_human_input(state: MessagesState):
    print("HUMAN INPUT REQUIRED FOR AUDIO TOOL FEEDBACK")
    human_message = interrupt("audio_human_input")
    return {
        "messages": [
            {
                "role": "human",
                "content": human_message
            }
        ]
    }
def storyboard_human_input(state: MessagesState):
    print("HUMAN INPUT REQUIRED FOR STORYBOARD TOOL FEEDBACK")
    human_message = interrupt("storyboard_human_input")
    return {
        "messages": [
            {
                "role": "human",
                "content": human_message
            }
        ]
    }
#workflow.add_node("video_human_input", video_human_input)
#workflow.add_node("audio_human_input", audio_human_input)
#workflow.add_node("storyboard_human_input", storyboard_human_input)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "supervisor")
# We now add a conditional edge
workflow.add_conditional_edges(
    "supervisor",
    go_next,
)
workflow.add_conditional_edges(
    "video_worker",
    check_video_tool_calls,
)
workflow.add_conditional_edges(
    "storyboard_worker",
    check_storyboard_tool_calls,
)
workflow.add_conditional_edges(
    "audio_worker",
    check_audio_tool_calls,
)
workflow.add_conditional_edges(
    "editor_worker",
    check_editor_tool_calls,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("video_tools", "video_worker")#"video_human_input")
#workflow.add_edge("video_human_input", "video_worker")

workflow.add_edge("storyboard_tools","storyboard_worker")#"storyboard_human_input")
#workflow.add_edge("storyboard_human_input", "storyboard_worker")

workflow.add_edge("audio_tools","audio_worker")#"audio_human_input");
#workflow.add_edge("audio_human_input","audio_worker");

workflow.add_edge("editor_tools","editor_worker");

In [10]:
# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)

generatingImg = False
generatingVid = False
# Use the agent
final_state = app.invoke(
    {"messages": [{"role": "user", "content": "Generate a *STORYBOARD*, *AUDIO*, *VIDEO* FOR ALL CHARACTERS THEN *EDIT* THE VIDEO AND AUDIO FILES TOGETHER for the following scene: SOPHIA(smacking his arm)Charles, how much have you beendrinking?CHARLES MONTAGUEWhy don’t you show our youngestfamily member around?Ethan, led by the lovely Sophia, takes in the surroundings.Match action to the same moment on the Family Reunion stage.INT. FAMILY REUNION SOUND STAGEEthan removes his goggles as the simulation freezes. He turnsto HENRY HILL, a Steve Jobs-type CEO of Family Reunion andspokesman of the infomercial.HENRYSo... what did you think?Ethan, visibly shaken.ETHANI’m literally blown away. It’sincredible—really incredible. Man,I’m so hooked.HENRYSo I guess you’ll be coming back,then? If you find any morematerial, keep it coming. The AIsimulations only get better andbetter.ETHANOh my God, you can count on it."}]},
    config={"configurable": {"thread_id": 42},"recursion_limit":100}
)
final_state["messages"][-1].content

supervisor
traveling
storyboard worker
check storyboard tool call
storyboarding
STORY ANALYSIS
------------------------------
The scene involves a family gathering where tension and curiosity coexist, scrutinizing the virtual reality experience. Charles Montague, seemingly enjoying himself a bit too much, has caught the attention of Sophia, who is slightly annoyed by his drinking. After Sophia smacks Charles on the arm, she encourages him to focus on showing their youngest family member, Ethan, around. The scene transitions to a highly advanced virtual reality stage known as the Family Reunion sound stage. Ethan, after removing his virtual-reality goggles, is left in awe. This simulation, designed by Henry Hill, a charismatic CEO likened to Steve Jobs, immerses users into a portrayal of family interactions. Henry is eager for feedback to continually enhance the simulations, to which Ethan responds with enthusiasm, confirming his intent to return and contribute further.
CHARACTERS
-----

'Routing to __end__'

In [ ]:
# Example usage:
source_video = 'staticAudioVid/0.mp4'
driving_video = '/Users/ryanlee/Documents/CreativeControl/LivePortrait/assets/examples/driving/d0.mp4'
output_video = 'CUSTOMOUTPUT/animated.mp4'

animate(source_video, driving_video, output_video)